In [8]:
import requests
import pandas as pd
import numpy as np
import json

In [31]:
from config import api_key

target_url = f'http://api.sportradar.us/nba/trial/v8/en/seasons/2022/REG/teams/583eca2f-fb46-11e1-82cb-f4ce4684ea4c/statistics.json?api_key={api_key}'

timberwolves_data = requests.get(target_url).json()

print(timberwolves_data)

{'id': '583eca2f-fb46-11e1-82cb-f4ce4684ea4c', 'name': 'Timberwolves', 'market': 'Minnesota', 'sr_id': 'sr:team:3426', 'reference': '1610612750', 'season': {'id': '5027b6ac-731c-4622-8d69-d863ae7c626b', 'year': 2022, 'type': 'REG'}, 'own_record': {'total': {'games_played': 82, 'minutes': 19830.0, 'field_goals_made': 3515, 'field_goals_att': 7167, 'field_goals_pct': 0.49, 'two_points_made': 2518, 'two_points_att': 4436, 'two_points_pct': 0.568, 'three_points_made': 997, 'three_points_att': 2731, 'three_points_pct': 0.365, 'blocked_att': 336, 'free_throws_made': 1467, 'free_throws_att': 1942, 'free_throws_pct': 0.755, 'offensive_rebounds': 749, 'defensive_rebounds': 2686, 'rebounds': 3435, 'assists': 2145, 'turnovers': 1212, 'assists_turnover_ratio': 1.77, 'steals': 657, 'blocks': 444, 'personal_fouls': 1768, 'tech_fouls': 79, 'points': 9494, 'fast_break_pts': 1208, 'flagrant_fouls': 12, 'points_off_turnovers': 1472, 'second_chance_pts': 1011, 'ejections': 9, 'foulouts': 15, 'points_in_p

In [9]:
print(json.dumps(timberwolves_data, indent=4))

{
    "id": "583eca2f-fb46-11e1-82cb-f4ce4684ea4c",
    "name": "Timberwolves",
    "market": "Minnesota",
    "sr_id": "sr:team:3426",
    "reference": "1610612750",
    "season": {
        "id": "5027b6ac-731c-4622-8d69-d863ae7c626b",
        "year": 2022,
        "type": "REG"
    },
    "own_record": {
        "total": {
            "games_played": 82,
            "minutes": 19830.0,
            "field_goals_made": 3515,
            "field_goals_att": 7167,
            "field_goals_pct": 0.49,
            "two_points_made": 2518,
            "two_points_att": 4436,
            "two_points_pct": 0.568,
            "three_points_made": 997,
            "three_points_att": 2731,
            "three_points_pct": 0.365,
            "blocked_att": 336,
            "free_throws_made": 1467,
            "free_throws_att": 1942,
            "free_throws_pct": 0.755,
            "offensive_rebounds": 749,
            "defensive_rebounds": 2686,
            "rebounds": 3435,
            "assis

In [18]:
timberwolves_data['players'][1]

{'id': '2e49c27a-06c5-4c4a-87fd-69840b783947',
 'full_name': 'Kyle Anderson',
 'first_name': 'Kyle',
 'last_name': 'Anderson',
 'position': 'F-G',
 'primary_position': 'SF',
 'jersey_number': '5',
 'sr_id': 'sr:player:608000',
 'reference': '203937',
 'total': {'games_played': 69,
  'games_started': 46,
  'minutes': 1957.0,
  'field_goals_made': 252,
  'field_goals_att': 495,
  'field_goals_pct': 0.509,
  'two_points_made': 209,
  'two_points_att': 390,
  'two_points_pct': 0.536,
  'three_points_made': 43,
  'three_points_att': 105,
  'three_points_pct': 0.41,
  'blocked_att': 19,
  'free_throws_made': 100,
  'free_throws_att': 136,
  'free_throws_pct': 0.735,
  'offensive_rebounds': 66,
  'defensive_rebounds': 302,
  'rebounds': 368,
  'assists': 335,
  'turnovers': 103,
  'assists_turnover_ratio': 3.25,
  'steals': 78,
  'blocks': 63,
  'personal_fouls': 146,
  'tech_fouls': 7,
  'points': 647,
  'flagrant_fouls': 1,
  'ejections': 1,
  'foulouts': 0,
  'true_shooting_att': 554.84,
 

In [15]:
timberwolves_df = pd.DataFrame(
    timberwolves_data['players']
)
timberwolves_df

,id,full_name,first_name,last_name,position,primary_position,jersey_number,sr_id,reference,total,average,name_suffix
0,042f45b2-1dda-4dce-a5ba-1e8b923178c1,Matt Ryan,Matt,Ryan,F,SF,37,sr:player:1158584,1630346,"{'games_played': 22, 'games_started': 0, 'minu...","{'minutes': 8.2, 'points': 3.4, 'off_rebounds'...",NaN
1,2e49c27a-06c5-4c4a-87fd-69840b783947,Kyle Anderson,Kyle,Anderson,F-G,SF,5,sr:player:608000,203937,"{'games_played': 69, 'games_started': 46, 'min...","{'minutes': 28.4, 'points': 9.4, 'off_rebounds...",NaN
2,34a159c8-9fe6-4386-bf01-3446f4248bcb,Austin Rivers,Austin,Rivers,G,SG,25,sr:player:607810,203085,"{'games_played': 52, 'games_started': 10, 'min...","{'minutes': 19.5, 'points': 4.9, 'off_rebounds...",NaN
3,37fbc3a5-0d10-4e22-803b-baa2ea0cdb12,Rudy Gobert,Rudy,Gobert,C,C,27,sr:player:607808,203497,"{'games_played': 70, 'games_started': 70, 'min...","{'minutes': 30.7, 'points': 13.4, 'off_rebound...",NaN
4,460b7264-b98f-483e-b841-59a18c2e4d67,Mike Conley,Mike,Conley,G,PG,10,sr:player:608194,201144,"{'games_played': 24, 'games_started': 24, 'min...","{'minutes': 31.4, 'points': 14.0, 'off_rebound...",NaN
5,4f80cdb1-1b10-4669-9774-045bc8a99c99,Taurean Prince,Taurean,Prince,F,PF,12,sr:player:996279,1627752,"{'games_played': 54, 'games_started': 4, 'minu...","{'minutes': 22.1, 'points': 9.1, 'off_rebounds...",NaN
6,59b8fe2b-cd77-4526-8047-816b1f081351,Nathan Knight,Nathan,Knight,F-C,PF,13,sr:player:1167592,1630233,"{'games_played': 38, 'games_started': 0, 'minu...","{'minutes': 7.7, 'points': 3.7, 'off_rebounds'...",NaN
7,6bc71e02-9a03-40db-aed4-332ab5193336,A.J. Lawson,A.J.,Lawson,G,SG,9,sr:player:1593864,1630639,"{'games_played': 1, 'games_started': 0, 'minut...","{'minutes': 2.0, 'points': 2.0, 'off_rebounds'...",NaN
8,6e16fada-c4ba-42af-ab1d-aca511a6c684,Jaylen Nowell,Jaylen,Nowell,G,SG,4,sr:player:1320568,1629669,"{'games_played': 65, 'games_started': 2, 'minu...","{'minutes': 19.3, 'points': 10.8, 'off_rebound...",NaN
9,823b2161-0c34-494c-9d7c-b438152f4f4d,Naz Reid,Naz,Reid,C-F,C,11,sr:player:1796560,1629675,"{'games_played': 68, 'games_started': 11, 'min...","{'minutes': 18.4, 'points': 11.5, 'off_rebound...",NaN


In [20]:
timberwolves_df['total']


0     {'games_played': 22, 'games_started': 0, 'minu...
1     {'games_played': 69, 'games_started': 46, 'min...
2     {'games_played': 52, 'games_started': 10, 'min...
3     {'games_played': 70, 'games_started': 70, 'min...
4     {'games_played': 24, 'games_started': 24, 'min...
5     {'games_played': 54, 'games_started': 4, 'minu...
6     {'games_played': 38, 'games_started': 0, 'minu...
7     {'games_played': 1, 'games_started': 0, 'minut...
8     {'games_played': 65, 'games_started': 2, 'minu...
9     {'games_played': 68, 'games_started': 11, 'min...
10    {'games_played': 28, 'games_started': 0, 'minu...
11    {'games_played': 79, 'games_started': 79, 'min...
12    {'games_played': 29, 'games_started': 29, 'min...
13    {'games_played': 43, 'games_started': 0, 'minu...
14    {'games_played': 15, 'games_started': 0, 'minu...
15    {'games_played': 25, 'games_started': 0, 'minu...
16    {'games_played': 79, 'games_started': 79, 'min...
17    {'games_played': 23, 'games_started': 0, '

In [28]:
totals = []
for row in timberwolves_df:
    totals = timberwolves_df['total']

totals

0     {'games_played': 22, 'games_started': 0, 'minu...
1     {'games_played': 69, 'games_started': 46, 'min...
2     {'games_played': 52, 'games_started': 10, 'min...
3     {'games_played': 70, 'games_started': 70, 'min...
4     {'games_played': 24, 'games_started': 24, 'min...
5     {'games_played': 54, 'games_started': 4, 'minu...
6     {'games_played': 38, 'games_started': 0, 'minu...
7     {'games_played': 1, 'games_started': 0, 'minut...
8     {'games_played': 65, 'games_started': 2, 'minu...
9     {'games_played': 68, 'games_started': 11, 'min...
10    {'games_played': 28, 'games_started': 0, 'minu...
11    {'games_played': 79, 'games_started': 79, 'min...
12    {'games_played': 29, 'games_started': 29, 'min...
13    {'games_played': 43, 'games_started': 0, 'minu...
14    {'games_played': 15, 'games_started': 0, 'minu...
15    {'games_played': 25, 'games_started': 0, 'minu...
16    {'games_played': 79, 'games_started': 79, 'min...
17    {'games_played': 23, 'games_started': 0, '

In [29]:
totals[1]

{'games_played': 69,
 'games_started': 46,
 'minutes': 1957.0,
 'field_goals_made': 252,
 'field_goals_att': 495,
 'field_goals_pct': 0.509,
 'two_points_made': 209,
 'two_points_att': 390,
 'two_points_pct': 0.536,
 'three_points_made': 43,
 'three_points_att': 105,
 'three_points_pct': 0.41,
 'blocked_att': 19,
 'free_throws_made': 100,
 'free_throws_att': 136,
 'free_throws_pct': 0.735,
 'offensive_rebounds': 66,
 'defensive_rebounds': 302,
 'rebounds': 368,
 'assists': 335,
 'turnovers': 103,
 'assists_turnover_ratio': 3.25,
 'steals': 78,
 'blocks': 63,
 'personal_fouls': 146,
 'tech_fouls': 7,
 'points': 647,
 'flagrant_fouls': 1,
 'ejections': 1,
 'foulouts': 0,
 'true_shooting_att': 554.84,
 'true_shooting_pct': 0.583,
 'efficiency': 1109,
 'points_off_turnovers': 98,
 'points_in_paint': 396,
 'points_in_paint_made': 198,
 'points_in_paint_att': 353,
 'points_in_paint_pct': 0.561,
 'effective_fg_pct': 0.553,
 'double_doubles': 10,
 'triple_doubles': 3,
 'fouls_drawn': 129,
 'of

In [27]:
for index, row in timberwolves_df['players'].iterrows():
    



                                                total
0   {'games_played': 22, 'games_started': 0, 'minu...
1   {'games_played': 69, 'games_started': 46, 'min...
2   {'games_played': 52, 'games_started': 10, 'min...
3   {'games_played': 70, 'games_started': 70, 'min...
4   {'games_played': 24, 'games_started': 24, 'min...
5   {'games_played': 54, 'games_started': 4, 'minu...
6   {'games_played': 38, 'games_started': 0, 'minu...
7   {'games_played': 1, 'games_started': 0, 'minut...
8   {'games_played': 65, 'games_started': 2, 'minu...
9   {'games_played': 68, 'games_started': 11, 'min...
10  {'games_played': 28, 'games_started': 0, 'minu...
11  {'games_played': 79, 'games_started': 79, 'min...
12  {'games_played': 29, 'games_started': 29, 'min...
13  {'games_played': 43, 'games_started': 0, 'minu...
14  {'games_played': 15, 'games_started': 0, 'minu...
15  {'games_played': 25, 'games_started': 0, 'minu...
16  {'games_played': 79, 'games_started': 79, 'min...
17  {'games_played': 23, 'ga

In [27]:
team_url = f'http://api.sportradar.us/nba/trial/v8/en/seasons/2022/REG/standings.json?api_key={api_key}'

team_data = requests.get(team_url).json()
def extract_team_data(data):
    teams = []
    if 'teams' in data:
        for team in data['teams']:
            team_id = team['id']
            team_name = team['name']
            teams.append({'id': team_id, 'name': team_name})
    if 'divisions' in data:
        for division in data['divisions']:
            teams += extract_team_data(division)
    if 'conferences' in data:
        for conference in data['conferences']:
            teams += extract_team_data(conference)
    return teams

# Extract team id and team name from the team_data dictionary
teams = extract_team_data(team_data)

# Create DataFrame from the extracted team data
df = pd.DataFrame(teams)

# Display the DataFrame
print(df)

                                      id           name
0   583eca88-fb46-11e1-82cb-f4ce4684ea4c      Grizzlies
1   583ecc9a-fb46-11e1-82cb-f4ce4684ea4c       Pelicans
2   583ecf50-fb46-11e1-82cb-f4ce4684ea4c      Mavericks
3   583ecb3a-fb46-11e1-82cb-f4ce4684ea4c        Rockets
4   583ecd4f-fb46-11e1-82cb-f4ce4684ea4c          Spurs
5   583ed0ac-fb46-11e1-82cb-f4ce4684ea4c          Kings
6   583ecfa8-fb46-11e1-82cb-f4ce4684ea4c           Suns
7   583ecdfb-fb46-11e1-82cb-f4ce4684ea4c       Clippers
8   583ec825-fb46-11e1-82cb-f4ce4684ea4c       Warriors
9   583ecae2-fb46-11e1-82cb-f4ce4684ea4c         Lakers
10  583ed102-fb46-11e1-82cb-f4ce4684ea4c        Nuggets
11  583eca2f-fb46-11e1-82cb-f4ce4684ea4c   Timberwolves
12  583ecfff-fb46-11e1-82cb-f4ce4684ea4c        Thunder
13  583ece50-fb46-11e1-82cb-f4ce4684ea4c           Jazz
14  583ed056-fb46-11e1-82cb-f4ce4684ea4c  Trail Blazers
15  583ecefd-fb46-11e1-82cb-f4ce4684ea4c          Bucks
16  583ec773-fb46-11e1-82cb-f4ce4684ea4c      Ca